In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2


In [2]:
from vpm_py.visualization import Visualizer, SliceFilter_3D, ValueFilter
from vpm_py.visualization import SliceStrategy
import os
from vpm_py.visualization import ResultPlot
import numpy as np
from vpm_py.vpm import VPM


In [ ]:
# Initialize VPM
vpm = VPM(
    number_of_equations=3,
    number_of_processors=1,
    rank=0,
    verbocity=2,
    dx_particle_mesh=0.1,
    dy_particle_mesh=0.1,
    dz_particle_mesh=0.1,
)


In [ ]:
z_slice = SliceFilter_3D(strategy=SliceStrategy.MAX_INTEGRAL, plane="Z")
y_slice = SliceFilter_3D(strategy=SliceStrategy.MAX_INTEGRAL, plane="Y")

plot_particles = ResultPlot(
    "particle",
    quantity="charge",
    component="magnitude",
    filters=[],
)
plot_mesh = ResultPlot(
    "mesh",
    quantity="charge",
    component="magnitude",
    filters=[],
    options={
        "s": "auto",
    },
)

plot_mesh_quantity_z = ResultPlot(
    "mesh",
    quantity="charge",
    component="magnitude",
    filters=[z_slice],
    options={
        "add_slice_plane": plot_particles,
    },
)
plot_mesh_quantity_y = ResultPlot(
    "mesh",
    quantity="velocity",
    component="magnitude",
    filters=[y_slice],
    options={
        "add_slice_plane": plot_particles,
    },
)

plotter = Visualizer(
    figure_size=(12, 10),
    plot_options=[
        plot_particles,
        plot_mesh,
        plot_mesh_quantity_z,
        plot_mesh_quantity_y,
    ],
)


In [ ]:
folder = "results_test/"
files = os.listdir(folder)
files_part = sorted([f for f in files if f.endswith("particles_test.h5")])
files_mesh = sorted([f for f in files if f.endswith("pm_test.h5")])
print(files_part)
print(files_mesh)
# total_frames = len(files_part)
# for idx in np.linspace(0, total_frames - 1, 30).astype(int):
#     file_mesh = os.path.join(folder, files_mesh[idx])
#     file_part = os.path.join(folder, files_part[idx])
#     plotter.load_results_from_disk(idx, total_frames, file_part, file_mesh)

plotter.animate_result_folder(
    folder,
    total_frames=len(files_part),
    start_frame=1,
    dt=0.1,
    save_filename="animation",
    format="mp4",
    particle_filename_format="particles_test.h5",
    mesh_filename_format="pm_test.h5",
)
